<h1>CS689A: Computational Linguistics for Indian Languages</h1>
<h2>Assignment-2</h2>
<h3>Question-2</h3>

**Dependencies:** `numpy, conllu, datasets, transformers, seqeval`

### Creating train-validation-test splits
> Replace the file accordingly

**Note: Skip this section if files are already available in form of train-val-test splits**

In [1]:
from conllu import parse

data_file = open("hi_hdtb-ud-train.conllu", "r", encoding="utf-8").read()
sentences = parse(data_file)
len(sentences)

13304

In [2]:
sentences = [sentence.serialize() for sentence in sentences]

In [3]:
from sklearn.model_selection import train_test_split

## Split size may be adjusted here
train, test = train_test_split(sentences,test_size=0.20)
train, val = train_test_split(train,test_size=0.10)

In [4]:
len(train),len(val),len(test)

(9578, 1065, 2661)

> Storing the outputs in `data/` folder

In [5]:
import os

## Create directory data/ if does not exist
if not os.path.exists('data'):
    os.makedirs('data')
    
for l,s in zip([train,val,test],['train','validation','test']):
    with open(f'data/{s}.conllu','w',encoding='utf-8') as fp:
        fp.write(''.join(l))

### Tokenization

The following section does tokenization for UPOS tagging task

> Loading data

In [6]:
from datasets import load_dataset

**Note: If the above convention for saving files was not followed, the file `ConlluIndic.py` needs to be editied so that `__TRAINING_FILE__` etc. point correctly**

In [7]:
dataset = load_dataset('ConlluIndic.py')
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset conllu_indic downloaded and prepared to C:/Users/HP/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/a4c9d43163d1876b550c10a1cb7e4a110ad131d1bf0fb3c0a5922a4efd642972. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 13304
    })
    validation: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 1659
    })
    test: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 1684
    })
})

> Tokenization

In [8]:
from transformers import AutoTokenizer

model_checkpoint = 'ai4bharat/indic-bert'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            #if label % 2 == 1:
            #    label += 1
            new_labels.append(label)

    return new_labels


In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["upos_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [11]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

  0%|          | 0/14 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [12]:
tokenized_datasets.save_to_disk('data/tokenized_datasets/')

### Fine-tuning and Evaluation

In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from datasets import load_metric, load_from_disk, load_dataset
import numpy as np
import torch

> Loads the model, tokenizer, data collator, metric and tokenized datasets

**Note: Don't forget to edit the file paths if the previous conventions were not followed**

In [14]:
model_checkpoint = 'ai4bharat/indic-bert'


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = load_from_disk('data/tokenized_datasets/')



data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = load_metric('seqeval')


dataset = load_dataset('ConlluIndic.py')

upos_feature = dataset['train'].features['upos_tags']
label_names = upos_feature.feature.names



id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}


model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,id2label=id2label,label2id=label2id)

C:\Users\HP\AppData\Local\Temp\ipykernel_17952\3837697733.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('seqeval')
Found cached dataset conllu_indic (C:/Users/HP/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/a4c9d43163d1876b550c10a1cb7e4a110ad131d1bf0fb3c0a5922a4efd642972)


  0%|          | 0/3 [00:00<?, ?it/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertForTokenClassification: ['sop_classifier.classifier.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.dense.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and a

> Metric computation

In [15]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


> Defining the trainer

`fp16 = True` may be set for GPU

Similarly `batch_size` can be increased

In [16]:
batch_size = 27
args = TrainingArguments(
    output_dir=f"model/upos",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=1,
    #fp16= True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    #eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


> Train the model

In [17]:
trainer.train()

C:\Users\HP\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13304
  Num Epochs = 2
  Instantaneous batch size per device = 27
  Total train batch size (w. parallel, distributed & accumulation) = 27
  Gradient Accumulation steps = 1
  Total optimization steps = 986
  Number of trainable parameters = 32866834
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.611727,0.777324,0.790884,0.784045,0.830763
2,0.993800,0.517005,0.804477,0.817231,0.810804,0.852626


***** Running Evaluation *****
  Num examples = 1659
  Batch size = 27
C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\H

TrainOutput(global_step=986, training_loss=0.7635843149305115, metrics={'train_runtime': 11521.201, 'train_samples_per_second': 2.309, 'train_steps_per_second': 0.086, 'total_flos': 86968698081996.0, 'train_loss': 0.7635843149305115, 'epoch': 2.0})

> Evaluate the model

**Note: Make sure evaluation results are reported on test dataset, for this training args should be changed**

In [18]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1659
  Batch size = 27


C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: PRON seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\HP\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171:

{'eval_loss': 0.5170047283172607,
 'eval_precision': 0.8044771346338343,
 'eval_recall': 0.817230849197583,
 'eval_f1': 0.8108038419390189,
 'eval_accuracy': 0.8526262753495403,
 'eval_runtime': 208.5838,
 'eval_samples_per_second': 7.954,
 'eval_steps_per_second': 0.297,
 'epoch': 2.0}